In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
cd /content/drive/MyDrive/AI/AICE/Tabular/Regression

In [ ]:
df = pd.read_csv('aapl.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
cond = (df['Open'].isnull() == True)
df[cond]

In [ ]:
df['Open'].interpolate(inplace = True)

In [ ]:
df['Volume'].astype('float')

In [ ]:
#cond = (df['Volume'].str.contains(',') == True)
df[cond].head()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.info()

In [ ]:
#dt.time, dt.day
cond = (df['Date'].dt.month == 12)
df[cond]

In [ ]:
#Date컬럼을 인덱스로 설정
df.set_index('Date', inplace = True, drop=True)
df.head()

In [ ]:
df.head()

In [ ]:
#df.loc['1984'].head()

In [ ]:
df.index.year[:5]

In [ ]:
df['Month'] = df.index.month.values

In [ ]:
df.head()


In [ ]:
df.tail(10)

In [ ]:
df.plot()

In [ ]:
df.plot(subplots=True)

In [ ]:
df.Volume.plot()

In [ ]:
df['Close'].rolling(5)

In [ ]:
df['Close'].rolling(5).mean()

In [ ]:
df['Close_5ma'] = df['Close'].rolling(5).mean()

In [ ]:
df.head(5)

In [ ]:
#df.loc['2007']['Close'].plot()
#df.loc['2007']['Close'].rolling(30).mean().plot()
#df.loc['2007'].rolling(window=30).mean()['Close'].plot()

In [ ]:
for i in range(1, 6):
  df['Close_{}'.format(i)] = df['Close'].shift(i)

In [ ]:
df.head(5)

In [ ]:
df.dropna(inplace=True,axis=0)

In [ ]:
df.head(5)

In [ ]:
#인덱스 뒤집기
#df_reversed = df.iloc[::-1]

In [ ]:
df.tail()


In [ ]:
#df_reversed.head()

In [ ]:
#df=df_reversed

In [ ]:
  #연도별 그룹화해서 시가, 종가 평균 보기
  df.groupby(df.index.year)[['Open', 'Close']].mean()

In [ ]:
#groupby 함수 대신 resample 함수 활용해서 그룹화하기
# T: min, H:  hours, D:day, M:monht, A:Annual

df.resample('A').mean()[['Open', 'Close']]
df.resample('2M').mean()[['Open', 'Close']]
df['Close'].resample('A').mean().plot(kind='bar')

In [ ]:
# 시가 월별 최대값 bar 차트
df.resample('M').max()['Open'].plot(kind='bar', figsize=(15,8))

In [ ]:
#df.loc['1999-01':'2002-01','Open'].resample('M').mean().plot(kind='bar')

In [ ]:
!pip install statsmodels

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
result=seasonal_decompose(df['Close'], model='additive', period=1)

In [ ]:
result.plot()

In [ ]:
#!pip install pmdarima==1.8.4
!pip install pmdarima

In [ ]:
from pmdarima import auto_arima

In [ ]:
model_arima = auto_arima(df['Close'], trace=True, error_action='ignore', start_p=1,start_q=1,max_p=3,max_q=3, suppress_warnings=True)
model_arima.fit(df['Close'])

In [ ]:

from statsmodels.tsa.arima.model import ARIMA
model=ARIMA(df['Close'],order=(0,1,1))
model=model.fit()

#model.plot_predict()
#plt.show()

# 예측
start = df.index[0]
end = df.index[-1]
pred = model.get_prediction(start=start, end=end, dynamic=False)
pred_mean = pred.predicted_mean
pred_ci = pred.conf_int()

# 원래 값과 예측 값의 최소값과 최대값을 구하여 y축 범위 설정
y_min = min(df['Close'].min(), pred_mean.min())
y_max = max(df['Close'].max(), pred_mean.max())

# 시각화
plt.figure(figsize=(10, 5))
plt.plot(df.index, df['Close'], label='Observed', color='blue')
plt.plot(pred_mean.index, pred_mean, label='Forecast', color='red')
plt.fill_between(pred_ci.index, pred_ci.iloc[:, 0], pred_ci.iloc[:, 1], color='pink', alpha=0.3)

plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Actual vs Predicted')
plt.ylim(y_min - (y_max - y_min) * 0.05, y_max + (y_max - y_min) * 0.05)  # y축 범위를 데이터의 최소/최대 값에 5% 여유를 둠
plt.legend()
plt.show()

In [ ]:
#7:3d으로 나누기
import tensorflow as tf
# 데이터프레임의 길이
n = len(df)

# 70%와 30% 인덱스 계산
train_size = int(n * 0.8)
valid_size = n - train_size

split_time = '2021-01-01'

# 데이터프레임 나누기
x_train = df[:split_time]['Close']
x_valid = df[split_time:]['Close']

print(x_train)
print(x_valid)

In [ ]:
x_train.shape,x_valid.shape

In [ ]:
#날짜를 기준으로 나누기
#split_time = '2008-10-06'
#x_train = df.loc[:split_time]['Close']
#x_valid = df.loc[split_time:]['Close']

In [ ]:
print("Train DataFrame:")
print(x_train)
print("\nValid DataFrame:")
print(x_valid)

In [ ]:

def windowed_Dataset(series, window_size, batch_size, shuffle_buffer):
  series = tf.expand_dims(series, axis=-1)
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(window_size + 1))
  ds = ds.shuffle(shuffle_buffer)
  ds = ds.map(lambda w: (w[:-1], w[-1]))
  return ds.batch(batch_size).prefetch(1)



In [ ]:
#윈도우 사이즈
window_size = 5
batch_size = 32
shuffle_buffer_size = 1000

In [ ]:
train_set = windowed_Dataset(x_train, window_size, batch_size, shuffle_buffer_size)
validation_set = windowed_Dataset(x_valid, window_size, batch_size, shuffle_buffer_size)


In [ ]:
train_set

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

EPOCHS = 20

es = EarlyStopping(monitor='val_loss', min_delta = 0, verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_mse', verbose=1, save_best_only=True, save_weights_only=True)

model= Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(None,1)))
model.add(LSTM(16,return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(8,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

history = model.fit(train_set, epochs=EPOCHS, validation_data=validation_set, callbacks=[es, mc])


In [ ]:
plt.figure(figsize=(12,9))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show

In [ ]:
predictions = model.predict(validation_set)


In [ ]:
print(predictions)

In [ ]:

print(x_valid)
# predictions의 길이와 x_valid의 길이가 다르므로, x_valid의 인덱스를 기준으로 predictions의 인덱스를 다시 설정
predictions_df = pd.DataFrame(predictions, columns=['Predictions'])
predictions_df.index = x_valid.index[5:19]

print(x_valid)

In [ ]:

# 예측값과 실제값을 그래프로 출력
plt.figure(figsize=(12, 6))

# 실제값을 그립니다.
plt.plot(x_valid, label='True Values', color='b')

# 예측값을 그립니다.
plt.plot(predictions_df, label='Predictions', color='r')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Predictions vs. True Values')
plt.legend()

plt.show()